<a href="https://colab.research.google.com/github/abhayraghuwanshi/Sentiment_classification/blob/main/phones_reviews_scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests as req
import bs4

In [2]:
header = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                    'AppleWebKit/537.36 (KHTML, like Gecko) '
                    'Chrome/51.0.2704.103 Safari/537.36'}

In [3]:
def search_amazon_product(query):
  """
  Args:
    query: str. The name of the product we are searching.

  Returns:
    result: dict . With asin numbers as keys and values as product name.
    Error : str. if no products founds
  """
  s_url =  "https://www.amazon.in/s?k="+query
  page = req.get(s_url, headers = header)
  print(s_url, page.status_code)
  if page.status_code == 200:
    results = {}
    soup = bs4.BeautifulSoup(page.content)
    for i in soup.findAll("div", {'class':"sg-col-20-of-24 s-result-item s-asin sg-col-0-of-12 sg-col-28-of-32 sg-col-16-of-20 sg-col sg-col-32-of-36 sg-col-12-of-16 sg-col-24-of-28"}):
      name =  i.findAll("span", {'class':"a-size-medium a-color-base a-text-normal"})
      results[i['data-asin']] = name[0].get_text()
    return results
  else:
      return "Error"

In [4]:
print(search_amazon_product('mobile'+ '&page=1'))

https://www.amazon.in/s?k=mobile&page=1 200
{'B07SDPJ4XJ': 'OPPO A5 2020 (Dazzling White, 4GB RAM, 64GB Storage) with No Cost EMI/Additional Exchange Offers', 'B07X4R63DF': 'Redmi 8A Dual (Sea Blue, 2GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery', 'B086KCCMCP': 'Samsung Galaxy M01 (Black, 3GB RAM, 32GB Storage) with No Cost EMI/Additional Exchange Offers', 'B07HGJJ559': 'Samsung Galaxy M21 (Midnight Blue, 4GB RAM, 64GB Storage)', 'B089MQ622N': 'Samsung Galaxy M01 Core (Black, 2GB RAM, 32GB Storage) with No Cost EMI/Additional Exchange Offers', 'B07WPVLKPW': 'Redmi 8A Dual (Sea Blue, 3GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery', 'B07X1KSLC3': 'Redmi 8A Dual (Midnight Grey, 2GB RAM, 32GB Storage) – Dual Cameras & 5,000 mAH Battery', 'B081FQ2X3T': 'itel A46 (Fiery Red, 2GB RAM, 16GB Storage)', 'B07SDPTP6Q': 'OPPO A5 2020 (Dazzling White, 3GB RAM, 64GB Storage) with No Cost EMI/Additional Exchange Offers', 'B07HGGYWL6': 'Samsung Galaxy M31 (Ocean Blue, 6GB RAM, 128

In [5]:
"""
for getting multiple page result

"""

asin_no_with_product_name = {}
for i in range(1,3):
  list1 = search_amazon_product('mobile'+ '&page=' + str(i))
  asin_no_with_product_name.update(list1)

https://www.amazon.in/s?k=mobile&page=1 200
https://www.amazon.in/s?k=mobile&page=2 200


In [6]:
def search_by_product_asni_numbers(asin):
  """
  Arg
    asin :str. asni no of product.
  Return
    str. review link of the product or Error if nothing found.
  """
  
  url = "https://www.amazon.in/dp/"+ str(asin)
  page =  req.get(url, headers=header)
  print(url, page.status_code)
  if page.status_code == 200:
    soup = bs4.BeautifulSoup(page.content)
    link = soup.findAll("a",{'data-hook':"see-all-reviews-link-foot"})
    return link[0]['href']
  else :
    return "Error"

In [7]:
print("the link of the reviews looks like this" , search_by_product_asni_numbers('B089MV96RW'))

https://www.amazon.in/dp/B089MV96RW 200
the link of the reviews looks like this /Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B089MV96RW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews


In [8]:
def get_reviews(review_link, page_number):
  """
  Arg:
    review_link: str.  link obtained from the search_by_product_asni_numbers.
    page_number: str| int . Page no of the review product.

  Return:
    list of reviews
  """

  url="https://www.amazon.in" + review_link + '&pageNumber='+ str(page_number)
  page=req.get(url, headers=header)
  if page.status_code==200:
    reviews = []
    soup = bs4.BeautifulSoup(page.content)
    for i in soup.findAll("span",{'data-hook':"review-body"}):
      reviews.append(i.text)
    return reviews
  else:
    return "Error"

In [9]:
get_reviews('/Samsung-Galaxy-Storage-Additional-Exchange/product-reviews/B089MV96RW/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews', 1)

["\n\n  Thank you samsung and Amazon for timely rakshabandhan delivery.Now for readers this is a review unbiased and written after testing and continuous usage :-Pro's1.Screen ppi is great as entry level device✌2.Battery backup is awesome thanks to samsungs R&D even with .7mamp charger.👀3.Android go version 10 is cute without craps4.Reading pdfs is easy and comfort for eyes atleast samsung has guts to say the truth.5.No lag till now because i dont need gaming other than that all regular even without go apps are nicely running.6.fingerprint and face lock i dont need so good they are not provided.7.Call quality is okay.Speaker is normal.8.For music lovers dowload any music app with equilizer will be fine.9.16gb/32gb and sd card is okay.10.Auto smart memory cleaner(use smart manager in settings once a day) for smooth RAM functioning and ROM cleaning🤗11. Supports OTG as well.Cons1. Camera below average still expected but samsung should work more on this front if they wanna beat chinese pho

In [10]:
# getting review of the product with the help of the asin numbers
reviews = []
for i in asin_no_with_product_name.keys():
  link = search_by_product_asni_numbers(i)
  for j in range(10):
    asmi_review = get_reviews(link, j)
    for x in asmi_review:
      reviews.append([i, x])

https://www.amazon.in/dp/B07X4R63DF 200
https://www.amazon.in/dp/B07HGJJ559 200
https://www.amazon.in/dp/B086KCCMCP 200
https://www.amazon.in/dp/B07HGGYWL6 200
https://www.amazon.in/dp/B07SDPJ4XJ 200
https://www.amazon.in/dp/B089MQ622N 200
https://www.amazon.in/dp/B07X1KSLC3 200
https://www.amazon.in/dp/B07WPVLKPW 200
https://www.amazon.in/dp/B08GCSL3VV 200
https://www.amazon.in/dp/B07X1KSWZ3 200
https://www.amazon.in/dp/B07DJCJBB3 200
https://www.amazon.in/dp/B07HGH8D2R 200
https://www.amazon.in/dp/B086KCDGTQ 200
https://www.amazon.in/dp/B07X3P1N3H 200
https://www.amazon.in/dp/B07SDPTP6Q 200
https://www.amazon.in/dp/B089MTL51J 200
https://www.amazon.in/dp/B07KXCKPZZ 200
https://www.amazon.in/dp/B081FQ2X3T 200
https://www.amazon.in/dp/B07HGH88GL 200
https://www.amazon.in/dp/B08698559Z 200
https://www.amazon.in/dp/B077PWBC78 200
https://www.amazon.in/dp/B087K78FD5 200
https://www.amazon.in/dp/B086977J48 200
https://www.amazon.in/dp/B07HGH8ML7 200
https://www.amazon.in/dp/B07KX1S161 200


In [11]:
import pandas as pd

In [12]:
reviews[0]

['B07X4R63DF',
 '\n\n  \xa0Defective product shipped.. reported within a  few  hours of receipt and no one showed up for inspection despite several follow ups. Now, it is not being replaced or refunded. Horrible experience!!\n\n']

In [13]:
review_data = pd.DataFrame(reviews)
review_data.head()

,0,1
0,B07X4R63DF,\n\n Defective product shipped.. reported wi...
1,B07X4R63DF,\n\n Nice phone camera better attractive look...
2,B07X4R63DF,\n\n Design-GoodCamera-NiceDisplay-satisfacto...
3,B07X4R63DF,\n\n The headphone jack is not working 😡 Des...
4,B07X4R63DF,"\n\n First sell, First order. Mobile is very ..."


In [14]:
review_data[0] = review_data[0].apply(lambda x: asin_no_with_product_name[x])
review_data.head()

,0,1
0,"Redmi 8A Dual (Sea Blue, 2GB RAM, 32GB Storage...",\n\n Defective product shipped.. reported wi...
1,"Redmi 8A Dual (Sea Blue, 2GB RAM, 32GB Storage...",\n\n Nice phone camera better attractive look...
2,"Redmi 8A Dual (Sea Blue, 2GB RAM, 32GB Storage...",\n\n Design-GoodCamera-NiceDisplay-satisfacto...
3,"Redmi 8A Dual (Sea Blue, 2GB RAM, 32GB Storage...",\n\n The headphone jack is not working 😡 Des...
4,"Redmi 8A Dual (Sea Blue, 2GB RAM, 32GB Storage...","\n\n First sell, First order. Mobile is very ..."


In [15]:
review_data.columns = ["phone_name", "review"]

In [19]:
review_data["review"] = review_data["review"].astype(str)
review_data["phone_name"] = review_data["phone_name"].astype(str)

In [20]:
review_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   phone_name  3000 non-null   object
 1   review      3000 non-null   object
dtypes: object(2)
memory usage: 47.0+ KB


In [30]:
review_data.to_excel('phones_reviews.xlsx',columns = ["phone_name", "review"],  index=False)     

In [29]:
review_data.to_csv("review_data_1.csv", columns = ["phone_name", "review"], index=False)